<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/main/scripts/100_note_generation/130_RAGIndexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Retrieval Augmented Generation of care notes

**Author:** Eva Rombouts  
**Date:** 2024-06-03  
**Updated:** 2024-07-01  
**Version:** 1.2

### Description
In this script the anonymous client notes generated [here](https://github.com/ekrombouts/GenCareAI/blob/main/scripts/100_GenerateAnonymousCareNotes.ipynb) are processed and stored in a vector database (Chroma). It enables querying of the database using OpenAI's embeddings and a retrieval-augmented generation system.

*Document Loading and Processing*: Documents are loaded from the Hugging Face platform, split into smaller sections by a LangChain Text Splitter, and pre-processed. The notes are split into smaller chunks, even though this probably was not necessary for this dataset. This step was taken for completeness to ensure scalability in the future.  
*Database Initialization and Population*: A Chroma vector database is initialized and populated it with the embedded document chunks.  
*Query Operations*: The RetrievalQA pipeline let's us search the database using natural language queries, demonstrating the capability to retrieve and display relevant information.

### Goal
My goal is to use this vector database to retrieve relevant examples for few-shot inference in prompts for creating synthetic client notes. This approach can help me improve the generation of this data by providing specific, contextually relevant examples that guide the model's results.

### Setup and configuration
- When running in CoLab Google Drive is mounted to persistently store the Chroma vector database.
- Retrieve API keys for OpenAI and HuggingFace, providing authentication for accessing the [embedding model](https://platform.openai.com/docs/guides/embeddings), the [QA model](https://platform.openai.com/docs/models) and the [dataset](https://huggingface.co/datasets/ekrombouts/dutch_nursing_home_reports). 

### Recommended Resources
- [RAG - Retrieval Augmented Generation](https://www.youtube.com/playlist?list=PL8motc6AQftn-X1HkaGG9KjmKtWImCKJS) with Sam Witteveen on YouTube
- [Python RAG Tutorial (with Local LLMs)](https://www.youtube.com/watch?v=2TJxpyO3ei4&t=323s) by Pixegami on YouTube
- And of course the [Langchain documentation](https://python.langchain.com/v0.1/docs/use_cases/question_answering/)

***Please note*** that the embedding isn't free. Embedding the 35.000+ notes costs appr $0.15. The costs for the examples of querying the database in this notebook are negligible.

Version 1.2: Term 'reports' changed to 'notes'

In [1]:
import os
# Determines the current environment (Google Colab or local)
def check_environment():
    try:
        import google.colab
        return "Google Colab"
    except ImportError:
        pass

    return "Local Environment"

In [2]:
# Installs and settings depending on the environment
# When running in CoLab, the Google drive is mounted and necessary packages are installed.
# Data paths are set and API keys retrieved

env = check_environment()

if env == "Google Colab":
    print("Running in Google Colab")
    !pip install -q langchain langchain-openai langchain-community chromadb datasets langchain-chroma
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    os.chdir('/content/drive/My Drive/Colab Notebooks/GenCareAI/scripts')
    OPENAI_API_KEY = userdata.get('GCI_OPENAI_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
else:
    print("Running in Local Environment")
    # !pip install -q chromadb datasets langchain-chroma
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('GCI_OPENAI_API_KEY')
    HF_TOKEN = os.getenv('HF_TOKEN')

Running in Local Environment


In [3]:
# Import necessary modules from Langchain and Hugging Face
import pandas as pd
# from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, OpenAI
from langchain_community.document_loaders import HuggingFaceDatasetLoader, DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.chains import RetrievalQA
from pprint import pprint

In [4]:
# Constants for dataset and storage paths
PATH_DATASET = '../../data/gcai_notes.csv' # Or use the HuggingFace dataset: 'ekrombouts/dutch_nursing_home_notes'
PATH_DB_GCAI = '../../data/chroma_db_gcai_notes'
COLLECTION_NAME = 'anonymous_notes'
MODEL = 'text-embedding-ada-002'

### Functions

In [5]:
def load_documents(path):
    """Load the dataset either from Hugging Face or a local CSV file based on the path provided."""
    
    try:
        # Try to load the Hugging Face dataset       
        loader = HuggingFaceDatasetLoader(path, page_content_column='note', use_auth_token=HF_TOKEN)
        return loader.load()
    
    except Exception:
        # If loading as a Hugging Face dataset fails, assume it's a CSV file        
        df = pd.read_csv(path)
        loader = DataFrameLoader(df, page_content_column='note')
        return loader.load()

In [6]:
def split_documents(documents: list[Document]):
  """Split large text documents into manageable chunks for better handling by ML models."""
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=800,
      chunk_overlap=100)
  chunks = text_splitter.split_documents(documents)
  # Index each chunk to maintain unique identifiers
  for idx, chunk in enumerate(chunks):
      chunk.metadata['id'] = str(idx)
  return chunks

In [7]:
def initialize_vectordb(persist_directory, embedding_function, collection_name):
    """Initialize the Chroma vector database, either loading an existing one or creating a new one."""
    if os.path.exists(persist_directory):
        return Chroma(persist_directory=persist_directory,
                      embedding_function=embedding_function,
                      collection_name=collection_name)
    else:
        return Chroma(embedding_function=embedding_function,
                      persist_directory=persist_directory,
                      collection_name=collection_name)


In [8]:
def load_existing_ids(vectordb):
    """Fetch existing document IDs from the database to avoid duplicates."""
    try:
        existing_items = vectordb.get(include=[])
        existing_ids = set(existing_items["ids"])
    except:
        existing_ids = set()
    return existing_ids

In [9]:
# Modified from https://github.com/pixegami/rag-tutorial-v2/blob/main/populate_database.py
def add_new_documents(vectordb, documents):
    """Add new documents to the database only if they don't already exist."""
    existing_ids = load_existing_ids(vectordb)
    print(f"Number of existing documents in DB: {len(existing_ids)}")
    # Only add documents that don't exist in the DB.
    new_documents = []
    for document in documents:
        if document.metadata["id"] not in existing_ids:
            new_documents.append(document)
    if len(new_documents):
        print(f"Adding new documents: {len(new_documents)}")
        new_document_ids = [document.metadata["id"] for document in new_documents]
        vectordb.add_documents(new_documents, ids=new_document_ids)
    else:
        print("No new documents to add")


### Embed and store texts

In [10]:
# Load, split and process documents
documents = load_documents(path=PATH_DATASET)
chunks = split_documents(documents=documents)

# # Consider experimenting with a smaller dataset
# # For large datasets you might need to split it in two
# documents_sample = documents[:20000]
# chunks = split_documents(documents=documents_sample)

print(len(documents))
print(len(chunks))

/Users/eva/anaconda3/envs/gcai2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/eva/anaconda3/envs/gcai2/lib/python3.12/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


8690
8723


In [13]:
# Initialize vector database, using OpenAI embeddings
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=MODEL)
vectordb = initialize_vectordb(PATH_DB_GCAI, embedding, COLLECTION_NAME)

# If you get an error, run this cell again.(TODO fix it)

In [14]:
# # Add new documents to the database
# def add_documents_in_batches(vectordb, documents, batch_size=1000):
#     """Add documents to the vector database in batches."""
#     for i in range(0, len(documents), batch_size):
#         batch = documents[i:i + batch_size]
#         add_new_documents(vectordb, batch)

# add_documents_in_batches(vectordb, chunks)

add_new_documents(vectordb, chunks)

Number of existing documents in DB: 8723
No new documents to add


### Query the database

In [ ]:
## To read the db from file

# vectordb = Chroma(persist_directory=FN_DB_GCAI,
#                   embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=MODEL),
#                   collection_name = COLLECTION_NAME
#                   )

In [ ]:
# Delete all items in the db

# items = vectordb.get(include=[])
# existing_ids = items["ids"]
# vectordb.delete(ids=existing_ids)

In [15]:
# Retrieve metadata and document IDs from the database
items = vectordb.get(include=['metadatas'])
existing_ids = set(items["ids"])
metadata = items['metadatas']
print(f"Number of existing documents in DB: {len(existing_ids)}")
print(metadata[0])

Number of existing documents in DB: 8723
{'id': '0', 'topic': 'ADL'}


In [16]:
# Set up a retriever for document querying
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [17]:
# Query the vector database using similarity search
query = 'gewichtsverlies'
docs = retriever.invoke(query)

print(f'Number of docs: {len(docs)}\n')
print(f'Retriever search type: {retriever.search_type}\n')

print(f'Documents most similar to "{query}":')
for doc in docs:
  print(doc.page_content)

Number of docs: 4

Retriever search type: similarity

Documents most similar to "gewichtsverlies":
Dringend overleg nodig met diëtist over voedingsaanpassingen dhr., ongewenst gewichtsverlies geconstateerd
Familie van Mevrouw bezorgd over haar gewichtsverlies. Overlegd met voedingsassistente voor verrijkte voedingssupplementen en extra observationele check-ups.
Familie van mevr spreekt zorgen uit over haar gewichtsverlies. Graag gewichtsmonitoring intensiveren en dieetvoorschriften herzien.
Familie van mevr meldt zorgen over gewichtsverlies, aandacht tijdens maaltijden gevraagd.


In [18]:
# Initialize the QA chain for answering questions using the retrieved documents
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(api_key=OPENAI_API_KEY),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [19]:
# Define a function to process the response
def process_llm_response(llm_response):
    print(100 * '*')
    print(f"\nresult: {llm_response['result']}")
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['id'], source.metadata['topic'])

In [20]:
# function to test the pipeline
def query_retrieval_pipeline(query):
  llm_response = qa_chain.invoke(query)
  pprint(llm_response)
  print(process_llm_response(llm_response))

In [21]:
query_retrieval_pipeline ("Wat moet je doen als je client afvalt in gewicht?")

{'query': 'Wat moet je doen als je client afvalt in gewicht?',
 'result': ' In alle gevallen is het belangrijk om een afspraak te maken voor '
           'overleg met een diëtist. Deze kan advies geven over voeding en '
           'eventueel voedingsaanpassingen doorvoeren om het gewichtsverlies '
           'tegen te gaan.',
 'source_documents': [Document(metadata={'id': '4733', 'topic': 'medisch_logistiek'}, page_content='Vandaag telefonisch contact gehad met dochter van cliënt, zij maakt zich zorgen over gewichtsverlies. Afspraak inplannen voor overleg en eventueel diëtist inschakelen.'),
                      Document(metadata={'id': '4576', 'topic': 'medisch_logistiek'}, page_content='Verpleging heeft opgemerkt dat meneer Jansen de laatste dagen moeite heeft met eten en drinken. Zijn gewicht lijkt ook af te nemen. Graag diëtist inschakelen voor advies.'),
                      Document(metadata={'id': '3960', 'topic': 'medisch_logistiek'}, page_content='Zoon van dhr. belt bezorgd 

In [22]:
query_retrieval_pipeline("Wat moet je doen als je client agressief gedrag vertoont?")

{'query': 'Wat moet je doen als je client agressief gedrag vertoont?',
 'result': ' Probeer de situatie te de-escaleren door rustig met de client te '
           'praten en hem/haar te begeleiden bij het omgaan met onverwachte '
           'situaties. Indien nodig kan ook de medicatie worden aangepast. Het '
           'is belangrijk om de veiligheid van zowel de client als het '
           'zorgpersoneel te waarborgen.',
 'source_documents': [Document(metadata={'id': '6572', 'topic': 'onrust'}, page_content='Mw reageert scherp: "Waarom bent u hier? Ga weg!" bij benadering door verzorgende. Vertoonde agressief gedrag door met kussen te gooien. Rustige omgeving geboden.'),
                      Document(metadata={'id': '6287', 'topic': 'onrust'}, page_content='Dhr is vanochtend erg agressief naar het zorgpersoneel toe, hij weigert zijn medicatie in te nemen en slaat om zich heen. Probeer de situatie te de-escaleren door rustig met hem te praten.'),
                      Document(metadat

In [23]:
query_retrieval_pipeline("Wat kan je doen als een cliënt onrustig is 's nachts?")

{'query': "Wat kan je doen als een cliënt onrustig is 's nachts?",
 'result': ' \n'
           '\n'
           'Overleg met het behandelteam, zoals een arts, psycholoog of '
           'nachtzorg, om te bepalen welke interventies of aanpassingen in de '
           'behandeling kunnen helpen om de onrust te verminderen en de cliënt '
           'beter te helpen.',
 'source_documents': [Document(metadata={'id': '4769', 'topic': 'medisch_logistiek'}, page_content="Dhr. onrustig 's nachts, overleg met arts of inzet van nachtzorg een optie is om hem beter te helpen."),
                      Document(metadata={'id': '4181', 'topic': 'medisch_logistiek'}, page_content='Mevr. uitte onrust tijdens de nacht. Observeren slaappatroon en eventueel overleggen met psycholoog.'),
                      Document(metadata={'id': '4146', 'topic': 'medisch_logistiek'}, page_content='Dhr. onrustig tijdens nacht, Overleg met psycholoog over mogelijke oorzaken en interventies om slaapproblemen te verminderen.

In [24]:
query_retrieval_pipeline("Welke interventies zijn ingezet voor het verbeteren van de nachtrust?")

{'query': 'Welke interventies zijn ingezet voor het verbeteren van de '
          'nachtrust?',
 'result': ' Er is extra aandacht besteed aan comfort en er zijn rustgevende '
           'maatregelen genomen. Mogelijk wordt er ook overlegd met een '
           'psycholoog voor mogelijke oorzaken en interventies om '
           'slaapproblemen te verminderen. Er wordt ook gekeken naar '
           'alternatieven voor slaap bevorderende middelen.',
 'source_documents': [Document(metadata={'id': '6890', 'topic': 'symptomen'}, page_content='Mw had vanavond een onrustige nacht en klaagde over nachtmerries. Extra aandacht besteed aan comfort en rustgevende maatregelen genomen.'),
                      Document(metadata={'id': '4647', 'topic': 'medisch_logistiek'}, page_content='Vragenlijst inzake slaappatroon ingevuld met mevr., lijkt op sommige dagen wat onrustiger te slapen. Mogelijk nachtrust verbetering bespreken.'),
                      Document(metadata={'id': '4146', 'topic': 'medisch

In [25]:
query_retrieval_pipeline("Wat zijn leuke dingen om te doen met bezoek?")

{'query': 'Wat zijn leuke dingen om te doen met bezoek?',
 'result': ' Een muzikaal optreden bijwonen, spelletjes spelen en bloemen '
           'ontvangen.',
 'source_documents': [Document(metadata={'id': '2879', 'topic': 'sociaal'}, page_content='U bezocht samen met andere bewoners een muzikaal optreden in de recreatiezaal, wat zorgde voor een gezellige middag.'),
                      Document(metadata={'id': '2132', 'topic': 'sociaal'}, page_content='In de middag was er een gezellige spelletjesmiddag in de recreatiezaal met enkele medebewoners. Dhr. deed enthousiast mee.'),
                      Document(metadata={'id': '2478', 'topic': 'sociaal'}, page_content='Mw. ontving vandaag bezoek van haar kleinkinderen en samen speelden ze een gezelschapsspel.'),
                      Document(metadata={'id': '2523', 'topic': 'sociaal'}, page_content='Tijdens het bezoekuur ontving Dhr. een prachtige bos bloemen van zijn kleindochter, wat een glimlach op zijn gezicht toverde.')]}
**********